Original code from:
https://colab.research.google.com/github/pytorch/pytorch.github.io/blob/master/assets/hub/pytorch_vision_googlenet.ipynb

# Get output probabilities

In [1]:
%%capture
import torch
import torchvision.models as models
from torchvision import transforms
import copy
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os 

# model = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True)
model = models.googlenet(pretrained=True)  #w/o arg, this will not pretrain it
# model = models.vgg16(pretrained=True)

model.eval() #set model in eval mode: https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch

# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [24]:
from google.colab import files
files.upload()

Saving car1.jpg to car1.jpg
Saving car1_noTires.jpg to car1_noTires.jpg
Saving car1_noTires_noHeadLights.jpg to car1_noTires_noHeadLights.jpg
Saving car1_noTires_noHeadLights_noPlate.jpg to car1_noTires_noHeadLights_noPlate.jpg


{'car1.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00d\x00d\x00\x00\xff\xdb\x00C\x00\x06\x04\x05\x05\x05\x04\x06\x05\x05\x05\x07\x06\x06\x07\t\x0f\n\t\x08\x08\t\x13\r\x0e\x0b\x0f\x16\x13\x17\x17\x16\x13\x15\x15\x18\x1b#\x1e\x18\x1a!\x1a\x15\x15\x1e)\x1f!$%\'(\'\x18\x1d+.+&.#&\'&\xff\xdb\x00C\x01\x06\x07\x07\t\x08\t\x12\n\n\x12&\x19\x15\x19&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&\xff\xc0\x00\x11\x08\x01H\x02H\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x01\x05\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x02\x03\x05\x06\x07\x00\x01\x08\xff\xc4\x00O\x10\x00\x01\x02\x04\x04\x03\x04\x06\x06\x07\x04\t\x03\x05\x01\x01\x01\x02\x03\x00\x04\x05\x11\x06\x12!1\x13AQ\x07"aq\x14#2R\x81\x91\x15Bb\xa1\xb1\xd13Sr\x82\x92\xc1\xe1\x16$C\xa2\x08\x17%4Dcs\x83\x93T\x94\xb2&5\xc2\xf0\xf1Ed\xff\xc4\x00\x1a\x01\x00\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\xff\xc4\x00)\x11\x00\x02\x02\x02\x02\x01\x03\x03\x05\x01\x01\

In [11]:
def get_output(model, input_image):
    # input_image = Image.open(input)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    return output
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    # print(output[0])  # The output has unnormalized scores. To get probabilities, you can run a softmax on it.

def output_probs(model, input_image):
    output = get_output(model, input_image)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    return probabilities

def get_top_categories(model, input_image, categories):
    probabilities = output_probs(model, input_image)
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())


In [25]:
car1 = Image.open('car1.jpg')
get_top_categories(model, car1, categories)

sports car 0.32742393016815186
convertible 0.1804104596376419
minivan 0.0995592251420021
beach wagon 0.05115921050310135
pickup 0.033875517547130585


In [26]:
# https://stackoverflow.com/questions/58496858/pytorch-runtimeerror-the-size-of-tensor-a-4-must-match-the-size-of-tensor-b
car1_noTires = Image.open('car1_noTires.jpg').convert('RGB')
get_top_categories(model, car1_noTires, categories)

sports car 0.426026314496994
convertible 0.08549865335226059
grille 0.054356131702661514
limousine 0.032750971615314484
minivan 0.031796686351299286


In [27]:
car1_noTires_noHeadLights = Image.open('car1_noTires_noHeadLights.jpg').convert('RGB')
get_top_categories(model, car1_noTires_noHeadLights, categories)

sports car 0.37603217363357544
convertible 0.14920584857463837
grille 0.0633411630988121
pickup 0.04799208790063858
racer 0.03867350518703461


In [30]:
car1_noTires_noHeadLights_noPlate = Image.open('car1_noTires_noHeadLights_noPlate.jpg').convert('RGB')
get_top_categories(model, car1_noTires_noHeadLights_noPlate, categories)

sports car 0.45262178778648376
convertible 0.15511702001094818
grille 0.09078475832939148
pickup 0.04729299619793892
racer 0.0443246066570282


In [31]:
car1_noTires_noHeadLights_noPlate_noWindows = Image.open('car1_noTires_noHeadLights_noPlate_noWindows.jpg').convert('RGB')
get_top_categories(model, car1_noTires_noHeadLights_noPlate_noWindows, categories)

home theater 0.1887952834367752
limousine 0.10566893965005875
convertible 0.057478856295347214
microwave 0.05330700799822807
entertainment center 0.04775479808449745


In [32]:
car1_noWindows = Image.open('car1_noWindows.jpg').convert('RGB')
get_top_categories(model, car1_noWindows, categories)

limousine 0.22650355100631714
convertible 0.19947753846645355
sports car 0.17881128191947937
minivan 0.037156108766794205
moving van 0.024462616071105003


In [33]:
car1_noWindows_noTires = Image.open('car1_noWindows_noTires.jpg').convert('RGB')
get_top_categories(model, car1_noWindows_noTires, categories)

limousine 0.4355795085430145
moving van 0.07499947398900986
trailer truck 0.045406240969896317
sports car 0.038989949971437454
convertible 0.029397061094641685


Strangely enough, some occlusions make the prob of a 'semi-correct' class go up. This is just like the unpredictability of adversarial examples.



---

In [34]:
image = Image.open('car_sketch.jpg').convert('RGB')
get_top_categories(model, image, categories)

plate rack 0.13989630341529846
studio couch 0.08316421508789062
fire screen 0.0807880312204361
mousetrap 0.048730336129665375
cradle 0.045956797897815704


This network is not good at detecting sketches